# Day 18 - claude

In [1]:
from heapq import heappush, heappop
from dataclasses import dataclass, field
from typing import List, Set, Tuple

@dataclass(order=True)
class PrioritizedNode:
    priority: int
    pos: tuple = field(compare=False)
    path: list = field(compare=False)
    cost: int = field(compare=False)

def manhattan_distance(pos1: Tuple[int, int], pos2: Tuple[int, int]) -> int:
    """Calculate Manhattan distance between two points."""
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def get_neighbors(pos: Tuple[int, int], grid_size: int) -> List[Tuple[int, int]]:
    """Get valid neighboring positions."""
    x, y = pos
    neighbors = []
    for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:  # right, down, left, up
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < grid_size and 0 <= new_y < grid_size:
            neighbors.append((new_x, new_y))
    return neighbors

def find_shortest_path(corrupted: Set[Tuple[int, int]], grid_size: int) -> int:
    """
    Find shortest path from (0,0) to (grid_size-1, grid_size-1) avoiding corrupted cells.
    Returns the minimum number of steps needed or -1 if no path exists.
    """
    start = (0, 0)
    goal = (grid_size - 1, grid_size - 1)
    
    # Check if start or goal is corrupted
    if start in corrupted or goal in corrupted:
        return -1
    
    visited = set()
    queue = []
    
    # Initial node: (priority, position, path_so_far, cost_so_far)
    initial_priority = manhattan_distance(start, goal)
    heappush(queue, PrioritizedNode(initial_priority, start, [start], 0))
    
    while queue:
        node = heappop(queue)
        current_pos = node.pos
        
        if current_pos == goal:
            return len(node.path) - 1  # Subtract 1 because path includes start position
        
        if current_pos in visited:
            continue
            
        visited.add(current_pos)
        
        for next_pos in get_neighbors(current_pos, grid_size):
            if next_pos not in visited and next_pos not in corrupted:
                new_path = node.path + [next_pos]
                new_cost = node.cost + 1
                new_priority = new_cost + manhattan_distance(next_pos, goal)
                heappush(queue, PrioritizedNode(new_priority, next_pos, new_path, new_cost))
    
    return -1  # No path found

def main():
    # Read input file
    corrupted = set()
    with open('input.txt', 'r') as f:
        # Only process first 1024 bytes (coordinates)
        for _ in range(1024):
            line = f.readline().strip()
            if not line:
                break
            x, y = map(int, line.split(','))
            corrupted.add((x, y))
    
    # Find shortest path in 71x71 grid (coordinates 0-70)
    result = find_shortest_path(corrupted, 71)
    print(f"Minimum steps needed: {result}")

if __name__ == "__main__":
    main()

Minimum steps needed: 380


## Part 2

In [2]:
from heapq import heappush, heappop
from dataclasses import dataclass, field
from typing import List, Set, Tuple, Optional

@dataclass(order=True)
class PrioritizedNode:
    priority: int
    pos: tuple = field(compare=False)
    path: list = field(compare=False)
    cost: int = field(compare=False)

def manhattan_distance(pos1: Tuple[int, int], pos2: Tuple[int, int]) -> int:
    """Calculate Manhattan distance between two points."""
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def get_neighbors(pos: Tuple[int, int], grid_size: int) -> List[Tuple[int, int]]:
    """Get valid neighboring positions."""
    x, y = pos
    neighbors = []
    for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:  # right, down, left, up
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < grid_size and 0 <= new_y < grid_size:
            neighbors.append((new_x, new_y))
    return neighbors

def path_exists(corrupted: Set[Tuple[int, int]], grid_size: int) -> bool:
    """
    Check if a path exists from (0,0) to (grid_size-1, grid_size-1) 
    avoiding corrupted cells.
    Uses A* pathfinding algorithm.
    """
    start = (0, 0)
    goal = (grid_size - 1, grid_size - 1)
    
    # Check if start or goal is corrupted
    if start in corrupted or goal in corrupted:
        return False
    
    visited = set()
    queue = []
    
    # Initial node
    initial_priority = manhattan_distance(start, goal)
    heappush(queue, PrioritizedNode(initial_priority, start, [start], 0))
    
    while queue:
        node = heappop(queue)
        current_pos = node.pos
        
        if current_pos == goal:
            return True
        
        if current_pos in visited:
            continue
            
        visited.add(current_pos)
        
        for next_pos in get_neighbors(current_pos, grid_size):
            if next_pos not in visited and next_pos not in corrupted:
                new_path = node.path + [next_pos]
                new_cost = node.cost + 1
                new_priority = new_cost + manhattan_distance(next_pos, goal)
                heappush(queue, PrioritizedNode(new_priority, next_pos, new_path, new_cost))
    
    return False

def find_blocking_byte(coordinates: List[Tuple[int, int]], grid_size: int) -> Optional[Tuple[int, int]]:
    """
    Find the first byte that blocks all paths from start to goal.
    Returns the coordinates of the blocking byte or None if no such byte exists.
    """
    corrupted = set()
    
    for coord in coordinates:
        corrupted.add(coord)
        if not path_exists(corrupted, grid_size):
            return coord
    
    return None

def main():
    # Read input file
    coordinates = []
    with open('input.txt', 'r') as f:
        for line in f:
            if not line.strip():
                break
            x, y = map(int, line.strip().split(','))
            coordinates.append((x, y))
    
    # Find the blocking byte in 71x71 grid (coordinates 0-70)
    blocking_byte = find_blocking_byte(coordinates, 71)
    
    if blocking_byte:
        print(f"{blocking_byte[0]},{blocking_byte[1]}")
    else:
        print("No blocking byte found")

if __name__ == "__main__":
    main()

26,50
